In [1]:
import pandas as pd
import numpy as np

#Import school data
new_school = pd.read_csv("data/New School Data - Sheet1.csv")

#Clean up school data
schoolnames = new_school.loc[new_school["Name"].str.contains("miles")]
schoolnames["id"]=range(1, len(schoolnames) + 1)
schoolloc = new_school.loc[new_school["Name"].str.contains("WA")]
schoolloc["id"]=range(1, len(schoolloc) + 1)
schoolloc=schoolloc.rename(columns={"Name": "Location"})
students = new_school.loc[new_school["Name"].str.contains("Students")]
students["id"]=range(1, len(students) + 1)
students=students.rename(columns={"Name": "student count"})

#Relabel grades for schools
grade = new_school.loc[new_school["Name"].str.contains("Grade")]
grade["id"]=range(1, len(grade) + 1)
grade=grade.rename(columns={"Name": "grades"})
grade.grades = grade.grades.apply(lambda x: "middle" if "Middle" in x else x)
grade.grades = grade.grades.apply(lambda x: "elementary" if "Primary" in x else x)
grade.grades = grade.grades.apply(lambda x: "high" if "High" in x else x)
grade.grades = grade.grades.apply(lambda x: "kindergarten" if "Kindergarten" in x else x)
grade.grades = grade.grades.apply(lambda x: "prek" if "Prekindergarten" in x else x)
grade.grades = grade.grades.apply(lambda x: "special education" if "Special" in x else x)
grade.grades = grade.grades.apply(lambda x: "middle" if "Regular School Grade: Grade 6 to Grade 8" in x else x)
grade.grades = grade.grades.apply(lambda x: "other" if "Other" in x else x)

#concat dataframes
waschools = pd.concat([schoolnames.set_index('id'),schoolloc.set_index('id'), grade.set_index('id'), students.set_index('id')], axis=1, join='inner')

#Create zipcode and student population columns
waschools["zip"] = waschools["Location"].str[-6:]
waschools[["student count", "count"]] = waschools["student count"].str.split(":", expand=True)
waschools=waschools.drop(columns=["student count", "Location"])

#Relabel performance column with ordinal values
waschools["performance"] = 0
waschools["performance"].iloc[0:111] = 5
waschools["performance"].iloc[112:214] = 4
waschools["performance"].iloc[214:287] = 3
waschools["performance"].iloc[287:337] = 2
waschools["performance"].iloc[337:395] = 1

#Make new dataframe for all schools ranked at 1 or more for performance
newwaschools = waschools.loc[waschools["performance"]!=0]

#Make new dataframe that shows average performance rating by zipcode
schoolavg = newwaschools.groupby("zip").mean("performance").reset_index().sort_values(by="performance", ascending = False)
schoolavg["zip"]=schoolavg["zip"].astype(int)


<ipython-input-1-4d3b3f6f5619>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schoolnames["id"]=range(1, len(schoolnames) + 1)
<ipython-input-1-4d3b3f6f5619>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schoolloc["id"]=range(1, len(schoolloc) + 1)
<ipython-input-1-4d3b3f6f5619>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [2]:
schoolavg

,zip,performance
40,98074,5.000000
41,98075,5.000000
46,98105,5.000000
10,98012,5.000000
26,98039,5.000000
...,...,...
55,98118,1.500000
69,98178,1.250000
67,98168,1.222222
59,98126,1.200000


In [3]:
waschools

,Name,grades,zip,count,performance
id,,,,,
1,Islander Middle School (18.7 miles),middle,98040,"1,059",5
2,Medina Elementary School (20.5 miles),elementary,98039,452,5
3,Wedgwood Elementary School (25.6 miles),elementary,98115,468,5
4,Smith Elementary (12.6 miles),elementary,98074,601,5
5,View Ridge Elementary School (24.4 miles),elementary,98115,596,5
...,...,...,...,...,...
501,Rainier View Elementary School (20.2 miles),elementary,98178,19,0
502,App At Lincoln (26.3 miles),elementary,98103,93,0
503,The Seattle Public Schools Skills Center (24.0...,high,98124,"1,217",0


In [4]:
waschools['Name'] = waschools['Name'].apply(lambda x: x[:-13])

In [5]:
waschools['Name']

id
1                        Islander Middle School
2                      Medina Elementary School
3                    Wedgwood Elementary School
4                              Smith Elementary
5                  View Ridge Elementary School
                         ...                   
501              Rainier View Elementary School
502                              App At Lincoln
503    The Seattle Public Schools Skills Center
504                           K-5 Stem At Boren
505            Handicapped Contractual Services
Name: Name, Length: 505, dtype: object

In [6]:
school_list = waschools.Name.values.tolist()
school_list

['Islander Middle School',
 'Medina Elementary School',
 'Wedgwood Elementary School',
 'Smith Elementary',
 'View Ridge Elementary School',
 'Federal Way Public Academy',
 'Aviation High School',
 'West Mercer Elementary',
 'Lakeridge Elementary School',
 'John Hay Elementary School',
 'Alcott Elementary',
 'Wilder Elementary',
 'Beaver Lake Middle School',
 'Newport Senior High School',
 'Somerset Elementary School',
 'Cherry Crest Elementary School',
 'Blackwell Elementary',
 'Timbercrest Junior High',
 'Pine Lake Middle School',
 'Inglewood Middle School',
 'Rosa Parks Elementary',
 'Mann Elementary',
 'East Ridge Elementary',
 'Skyline High School',
 'Wellington Elementary',
 'Cascade Ridge Elementar',
 'Island Park Elementary',
 'Carson Elementary',
 'Clyde Hill Elementary',
 'Bryant Elementary School',
 'Laurelhurst Elementary School',
 'Mcauliffe Elementary',
 'Issaquah High Schoo',
 'Whittier Elementary School',
 'Endeavour Elementary Schoo',
 'Glacier Park Elementary',
 'Merc

Data from https://data.wa.gov/education/Report-Card-Assessment-Data-2021-22-School-Year/v928-8kke

In [7]:
rc = pd.read_csv("data/Report_Card_Assessment_Data_2021-22_School_Year.csv")

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
rc.head()

,SchoolYear,OrganizationLevel,County,ESDName,ESDOrganizationId,DistrictCode,DistrictName,DistrictOrganizationId,SchoolCode,SchoolName,...,Count of students expected to test including previously passed,CountMetStandard,PercentMetStandard,PercentLevel1,PercentLevel2,PercentLevel3,PercentLevel4,PercentMetTestedOnly,PercentNoScore,DataAsOf
0,2021-22,District,Grays Harbor,Capital Region ESD 113,100004.0,14005.0,Aberdeen School District,100010.0,NaN,District Total,...,176.0,54.0,30.7%,0.460227,0.221591,0.198864,0.107955,0.310345,0.011364,9/7/2022
1,2021-22,District,Grays Harbor,Capital Region ESD 113,100004.0,14005.0,Aberdeen School District,100010.0,NaN,District Total,...,107.0,35.0,32.7%,0.420561,0.224299,0.196262,0.130841,0.336538,0.028037,9/7/2022
2,2021-22,District,Grays Harbor,Capital Region ESD 113,100004.0,14005.0,Aberdeen School District,100010.0,NaN,District Total,...,111.0,39.0,35.1%,0.441441,0.198198,0.198198,0.153153,0.354545,0.009009,9/7/2022
3,2021-22,District,Grays Harbor,Capital Region ESD 113,100004.0,14005.0,Aberdeen School District,100010.0,NaN,District Total,...,NaN,NaN,Suppressed: N<10,NaN,NaN,NaN,NaN,NaN,NaN,9/7/2022
4,2021-22,District,Grays Harbor,Capital Region ESD 113,100004.0,14005.0,Aberdeen School District,100010.0,NaN,District Total,...,NaN,NaN,Suppressed: N<10,NaN,NaN,NaN,NaN,NaN,NaN,9/7/2022


In [9]:
rc["SchoolCode"].describe()

count    587239.000000
mean       3599.351349
std        1097.526505
min        1502.000000
25%        2756.000000
50%        3471.000000
75%        4392.000000
max        5953.000000
Name: SchoolCode, dtype: float64

In [10]:
rc["CurrentSchoolType"].value_counts()

P    514554
A     54177
S      6445
R      4844
J      2415
I      2208
Q      1936
Z       404
T       256
Name: CurrentSchoolType, dtype: int64

In [11]:
rc = rc.drop(columns=["SchoolYear", "ESDName", "ESDOrganizationId", "DistrictCode", "DistrictOrganizationId", 
                    "Count of students expected to test including previously passed",
                     "DataAsOf", "SchoolOrganizationId", "CurrentSchoolType", "TestAdministration", 
                      "Count of Students Expected to Test", "Suppression", "StudentGroupType", "StudentGroup",
                     ])
rc = rc[rc["County"] == "King"].drop(columns=["County"])

In [12]:
rcs = rc[rc["OrganizationLevel"] == "School"].drop(columns=["OrganizationLevel"])
rcs

,DistrictName,SchoolCode,SchoolName,GradeLevel,TestSubject,CountMetStandard,PercentMetStandard,PercentLevel1,PercentLevel2,PercentLevel3,PercentLevel4,PercentMetTestedOnly,PercentNoScore
14541,Seattle School District No. 1,1579.0,Tops K-8 School,07,Math,36.0,83.7%,0.046512,0.116279,0.348837,0.488372,0.837209,0.000000
14542,Seattle School District No. 1,1579.0,Tops K-8 School,07,Math,16.0,76.2%,0.095238,0.142857,0.380952,0.380952,0.761905,0.000000
14543,Seattle School District No. 1,1579.0,Tops K-8 School,07,Math,25.0,67.6%,0.108108,0.189189,0.297297,0.378378,0.694444,0.027027
14544,Seattle School District No. 1,1579.0,Tops K-8 School,07,Math,NaN,Suppressed: N<10,NaN,NaN,NaN,NaN,NaN,NaN
14545,Seattle School District No. 1,1579.0,Tops K-8 School,07,Math,NaN,71.7%,0.094340,0.169811,0.301887,0.415094,0.730769,0.018868
...,...,...,...,...,...,...,...,...,...,...,...,...,...
759445,Auburn School District,5664.0,Auburn Online,All Grades,Math,8.0,7.1%,0.336283,0.176991,0.044248,0.026549,0.121212,0.415929
759446,Auburn School District,5664.0,Auburn Online,All Grades,Science,6.0,35.3%,0.117647,0.117647,0.176471,0.176471,0.600000,0.411765
759447,Auburn School District,5664.0,Auburn Online,All Grades,Science,8.0,40.0%,0.100000,0.150000,0.300000,0.100000,0.615385,0.350000
759448,Auburn School District,5664.0,Auburn Online,All Grades,Science,7.0,15.9%,0.181818,0.113636,0.068182,0.090909,0.350000,0.545455


In [13]:
rcs = rcs[rcs["GradeLevel"] == "All Grades"].drop(columns=["GradeLevel", "DistrictName", "PercentLevel1",
                                                              "PercentLevel2", "PercentLevel3", "PercentLevel4",
                                                              "PercentMetTestedOnly", "PercentNoScore", "CountMetStandard"])

In [14]:
rcs["PercentMetStandard"]=rcs["PercentMetStandard"].replace("Suppressed: N<10", "0.0%")
rcs["PercentMetStandard"]=rcs["PercentMetStandard"].replace("No Students", "0.0%")

In [15]:
rcs["PercentMetStandard"] = rcs["PercentMetStandard"].apply(lambda x: x.strip("%"))
rcs["PercentMetStandard"] = rcs["PercentMetStandard"].apply(lambda x: x.strip(">"))
rcs["PercentMetStandard"] = rcs["PercentMetStandard"].apply(lambda x: x.strip("<"))

In [16]:
rcs["PercentMetStandard"] = rcs["PercentMetStandard"].astype(float)

In [17]:
rcs = rcs.groupby(["SchoolName"]).aggregate({"PercentMetStandard":"mean"}).reset_index()

In [18]:
rcs

,SchoolName,PercentMetStandard
0,Adams Elementary School,50.401493
1,Adelaide Elementary School,22.562393
2,Aki Kurose Middle School,16.193913
3,Alan T. Sugiyama High School,2.573333
4,Albert Einstein Elementary,36.756522
...,...,...
500,Woodinville Community Center,0.000000
501,Woodinville HS,28.024528
502,Woodmont K-8 School,26.505479
503,Woodmoor Elementary,26.248696


In [31]:
rcs["rating"] = rcs["PercentMetStandard"]/10
rcs=rcs.drop(columns=["PercentMetStandard"])

In [32]:
rcs

,SchoolName,rating
0,Adams Elementary School,5.040149
1,Adelaide Elementary School,2.256239
2,Aki Kurose Middle School,1.619391
3,Alan T. Sugiyama High School,0.257333
4,Albert Einstein Elementary,3.675652
...,...,...
500,Woodinville Community Center,0.000000
501,Woodinville HS,2.802453
502,Woodmont K-8 School,2.650548
503,Woodmoor Elementary,2.624870


In [20]:
rcs["rating"].describe()

count    505.000000
mean       3.160548
std        1.764996
min        0.000000
25%        1.868455
50%        2.965915
75%        4.521972
max        7.575079
Name: rating, dtype: float64

From https://eds.ospi.k12.wa.us/directoryeds.aspx

In [21]:
addresses=pd.read_csv("data/Washington_School_Directory_20230214.csv")

In [22]:
addresses.head()

,ESDCode,ESDName,LEACode,LEAName,SchoolCode,SchoolName,LowestGrade,HighestGrade,AddressLine1,AddressLine2,City,State,ZipCode,PrincipalName,Email,Phone,OrgCategoryList,AYPCode,GradeCategory
0,17801,Puget Sound Educational Service District 121,27010,Tacoma School District,2772,Fawcett Elementary School,PK,5,3702 McKinley Ave,NaN,Tacoma,Washington,98404-0000,Emily Hathaway,ehathaw@Tacoma.K12.Wa.US,253.571.4700,"Public School, Regular School",P,Elementary School
1,17801,Puget Sound Educational Service District 121,27010,Tacoma School District,2805,Lowell Elementary School,K,5,810 Mr. Dahl DR,NaN,Tacoma,Washington,98403-0000,Joshua Benedict,jbenedi@tacoma.k12.wa.us,253.571.7200,"Public School, Regular School",P,Elementary School
2,17801,Puget Sound Educational Service District 121,27010,Tacoma School District,2806,Reed Elementary School,PK,5,1802 S 36th St,NaN,Tacoma,Washington,98418-0000,Abby Sloan,asloan@Tacoma.K12.Wa.US,253.571.6464,"Public School, Regular School",P,Elementary School
3,17801,Puget Sound Educational Service District 121,27010,Tacoma School District,2871,Edison Elementary School,PK,5,5830 S Pine St,NaN,Tacoma,Washington,98409-0000,Anita Roth,aroth@Tacoma.K12.Wa.US,253.571.1700,"Public School, Regular School",P,Elementary School
4,17801,Puget Sound Educational Service District 121,27010,Tacoma School District,2872,Browns Point Elementary School,K,5,1526 51st St Ne,NaN,Tacoma,Washington,98422-0000,Cynthia Horner,CHORNER@tacoma.k12.wa.us,253.571.7600,"Public School, Regular School",P,Elementary School


In [23]:
addresses = addresses[["ESDCode", "LEACode", "LEAName", "SchoolCode", "SchoolName", "ZipCode"]]

In [24]:
addresses["ZipCode"] = addresses["ZipCode"].apply(lambda x: x[:5])

In [25]:
addresses

,ESDCode,LEACode,LEAName,SchoolCode,SchoolName,ZipCode
0,17801,27010,Tacoma School District,2772,Fawcett Elementary School,98404
1,17801,27010,Tacoma School District,2805,Lowell Elementary School,98403
2,17801,27010,Tacoma School District,2806,Reed Elementary School,98418
3,17801,27010,Tacoma School District,2871,Edison Elementary School,98409
4,17801,27010,Tacoma School District,2872,Browns Point Elementary School,98422
...,...,...,...,...,...,...
2551,29801,31311,Sultan School District,4274,Sultan Senior High School,98294
2552,29801,31311,Sultan School District,4399,Gold Bar Elementary,98251
2553,29801,31330,Darrington School District,3188,Darrington High School,98241
2554,29801,31330,Darrington School District,3609,Darrington Elementary School,98241


In [26]:
zips=[98052,98115,98103,98092,98042,98023,98105,98003,98034,98118,98133,98058,98125,98031,98122,98033,98006,98032,98030,98004,98001,98059,98117,98056,98198,98168,98155,98027,98109,98002,98038,98144,98146,98074,98119,98007,98040,98029,98107,98011,98108,98072,98106,98178,98116,98112,98008,98053,98028,98126,98188,98102,98055,98075,98177,98022,98166,98199,98005,98121,98136,98045,98057,98019,98104,98077,98101,98065,98148,98070,98014,98354,98024,98051,98010,98047,98251,98134,98039,98195,98158,98068,98256,98288,98050,98132,98171,98184,98054,98009,98013,98015,98025,98035,98041,98062,98064,98063,98071,98073,98083,98089,98093,98111,98113,98114,98124,98127,98131,98129,98139,98138,98141,98145,98151,98154,98160,98164,98161,98165,98170,98174,98175,98181,98185,98190,98194,98191,98224,98471,98481]
zips_strings = [str(i) for i in zips]
zips = zips_strings

In [27]:
non_waaddresses = addresses[~addresses['ZipCode'].isin(zips)]
addresses.drop(index = non_waaddresses.index, inplace = True, axis = 0)


In [29]:
addresses=addresses[["SchoolName", "ZipCode"]]

In [30]:
addresses

,SchoolName,ZipCode
142,Discovery Primary School,98354
143,Surprise Lake Middle School,98354
984,Middle College High School,98125
985,Tops K-8 School,98102
986,Seattle World School,98122
...,...,...
2357,Bellevue Digital Discovery,98005
2360,Willow Crest Elementary,98092
2370,Kent Virtual Academy,98030
2519,Index Elementary School,98256


In [33]:
schoolzip = rcs.merge(addresses, left_on="SchoolName", right_on="SchoolName")
schoolzip

,SchoolName,rating,ZipCode
0,Adams Elementary School,5.040149,98107
1,Adelaide Elementary School,2.256239,98023
2,Aki Kurose Middle School,1.619391,98118
3,Alan T. Sugiyama High School,0.257333,98118
4,Albert Einstein Middle School,2.578268,98177
...,...,...,...
456,Woodin Elementary,5.014844,98011
457,Woodinville HS,2.802453,98072
458,Woodmont K-8 School,2.650548,98198
459,Woodmoor Elementary,2.624870,98011
